# Project 03 — CRR Binomial Tree & Discrete Delta Hedging

**Goal**
- Build a **Cox–Ross–Rubinstein (CRR)** binomial tree.
- Price **European Call/Put** by backward induction.
- Visualize the **tree** (interactive Plotly).
- Simulate **discrete-time delta hedging** and analyze **replication error**.
- Compare binomial price to **Black–Scholes** and show **convergence**.

> Everything (code + explanations) is in **English**.

## 0) Setup (paths + reproducibility)
Creates an `assets/` folder next to this notebook to store interactive HTML charts.

In [1]:
from __future__ import annotations

from pathlib import Path
import numpy as np

SEED = 42
rng = np.random.default_rng(SEED)

PROJECT_DIR = Path.cwd()
ASSETS_DIR = PROJECT_DIR / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", PROJECT_DIR)
print("ASSETS_DIR:", ASSETS_DIR.resolve())

CWD: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\03_binomial_discrete_hedging
ASSETS_DIR: C:\Users\Karim\Desktop\quant-finance-portfolio\projects\03_binomial_discrete_hedging\assets


## 1) Imports

In [2]:
import math
import pandas as pd

from scipy.stats import norm

import plotly.graph_objects as go
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display, clear_output

## 2) CRR model (short math)

Time step: $\Delta t = T/N$

CRR up/down factors:
$$
u = e^{\sigma\sqrt{\Delta t}},\qquad d = \frac{1}{u}
$$

Risk-neutral probability:
$$
p = \frac{e^{r\Delta t}-d}{u-d}
$$

Stock at node $(i,j)$:
$$
S_{i,j} = S_0\,u^j d^{\,i-j}
$$

European payoff (maturity):
- Call: $\max(S_{N,j}-K,0)$
- Put:  $\max(K-S_{N,j},0)$

Backward induction:
$$
V_{i,j} = e^{-r\Delta t}\big(pV_{i+1,j+1} + (1-p)V_{i+1,j}\big)
$$

Node delta:
$$
\Delta_{i,j} = \frac{V_{i+1,j+1}-V_{i+1,j}}{S_{i+1,j+1}-S_{i+1,j}}
$$

## 3) CRR implementation (stock tree + option pricing + node deltas)

In [3]:
def crr_params(T: float, N: int, r: float, sigma: float):
    if N <= 0:
        raise ValueError("N must be >= 1")
    if T <= 0:
        raise ValueError("T must be > 0")
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    dt = T / N
    u = math.exp(sigma * math.sqrt(dt))
    d = 1.0 / u
    disc = math.exp(-r * dt)
    p = (math.exp(r * dt) - d) / (u - d)
    if not (0.0 < p < 1.0):
        raise ValueError(f"Risk-neutral probability p={p:.6f} not in (0,1). Check params.")
    return dt, u, d, p, disc

def build_stock_tree(S0: float, u: float, d: float, N: int) -> np.ndarray:
    S = np.full((N + 1, N + 1), np.nan, dtype=float)
    S[0, 0] = S0
    for i in range(1, N + 1):
        for j in range(0, i + 1):
            S[i, j] = S0 * (u ** j) * (d ** (i - j))
    return S

def price_european_binomial(
    S0: float, K: float, T: float, r: float, sigma: float, N: int, option_type: str = "call"
) -> dict:
    dt, u, d, p, disc = crr_params(T, N, r, sigma)
    S = build_stock_tree(S0, u, d, N)

    V = np.full_like(S, np.nan)
    Delta = np.full_like(S, np.nan)

    if option_type.lower() == "call":
        V[N, :N+1] = np.maximum(S[N, :N+1] - K, 0.0)
    elif option_type.lower() == "put":
        V[N, :N+1] = np.maximum(K - S[N, :N+1], 0.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    for i in range(N - 1, -1, -1):
        for j in range(0, i + 1):
            V[i, j] = disc * (p * V[i + 1, j + 1] + (1.0 - p) * V[i + 1, j])

            Su = S[i + 1, j + 1]
            Sd = S[i + 1, j]
            Vu = V[i + 1, j + 1]
            Vd = V[i + 1, j]
            Delta[i, j] = (Vu - Vd) / (Su - Sd)

    return {
        "price": float(V[0, 0]),
        "S": S,
        "V": V,
        "Delta": Delta,
        "dt": dt,
        "u": u,
        "d": d,
        "p": p,
        "disc": disc,
        "N": int(N),
    }

## 4) Black–Scholes reference (for convergence)

In [4]:
def bs_d1_d2(S0: float, K: float, T: float, r: float, sigma: float):
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return d1, d2

def bs_price(S0: float, K: float, T: float, r: float, sigma: float, option_type: str = "call") -> float:
    d1, d2 = bs_d1_d2(S0, K, T, r, sigma)
    if option_type.lower() == "call":
        return S0 * norm.cdf(d1) - K * math.exp(-r*T) * norm.cdf(d2)
    elif option_type.lower() == "put":
        return K * math.exp(-r*T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

def bs_delta(S0: float, K: float, T: float, r: float, sigma: float, option_type: str = "call") -> float:
    d1, _ = bs_d1_d2(S0, K, T, r, sigma)
    if option_type.lower() == "call":
        return float(norm.cdf(d1))
    elif option_type.lower() == "put":
        return float(norm.cdf(d1) - 1.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

## 5) Pricing sanity check (CRR vs BS)

**Interpretation**
- For large **N**, the CRR price should approach the BS price.

In [5]:
S0 = 100.0
K  = 100.0
T  = 1.0
r  = 0.03
sigma = 0.20
option_type = "call"

bs_ref = bs_price(S0, K, T, r, sigma, option_type)
res = price_european_binomial(S0, K, T, r, sigma, N=200, option_type=option_type)

print("Black–Scholes reference:", bs_ref)
print("Binomial (N=200):", res["price"])
print("Absolute error:", abs(res["price"] - bs_ref))
print("CRR params:", "dt=", res["dt"], "u=", res["u"], "d=", res["d"], "p=", res["p"])

Black–Scholes reference: 9.413403383853016
Binomial (N=200): 9.403493192324275
Absolute error: 0.009910191528740953
CRR params: dt= 0.005 u= 1.0142426086996437 d= 0.9859573946337119 p= 0.501768046858978


## 6) Convergence plot: Binomial price vs N

**Interpretation**
- As **N increases**, discretization becomes finer and the CRR lattice converges to BS.

In [6]:
Ns = [5, 10, 25, 50, 75, 100, 150, 200, 300, 400, 600, 800]
prices = [price_european_binomial(S0, K, T, r, sigma, N=n, option_type=option_type)["price"] for n in Ns]

df_conv = pd.DataFrame({"N": Ns, "binomial_price": prices})
df_conv["bs_price"] = bs_ref
df_conv["abs_error"] = (df_conv["binomial_price"] - bs_ref).abs()
df_conv

,N,binomial_price,bs_price,abs_error
0,5,9.796472,9.413403,0.383069
1,10,9.217851,9.413403,0.195552
2,25,9.489075,9.413403,0.075672
3,50,9.373839,9.413403,0.039564
4,75,9.438554,9.413403,0.025151
5,100,9.393596,9.413403,0.019808
6,150,9.400193,9.413403,0.013211
7,200,9.403493,9.413403,0.009910
8,300,9.406795,9.413403,0.006608
9,400,9.408447,9.413403,0.004957


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_conv["N"], y=df_conv["binomial_price"], mode="lines+markers", name="Binomial (CRR)"))
fig.add_trace(go.Scatter(x=df_conv["N"], y=df_conv["bs_price"], mode="lines", name="Black–Scholes"))
fig.update_layout(
    title="Convergence: CRR binomial price → Black–Scholes",
    xaxis_title="N (number of steps)",
    yaxis_title="Option price",
    template="plotly_dark"
)
fig.show()
fig.write_html(ASSETS_DIR / "convergence_binomial_vs_bs.html")

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_conv["N"], y=df_conv["abs_error"], mode="lines+markers", name="|CRR - BS|"))
fig2.update_layout(
    title="Absolute error vs N (log-x)",
    xaxis_title="N",
    yaxis_title="Absolute error",
    template="plotly_dark"
)
fig2.update_xaxes(type="log")
fig2.show()
fig2.write_html(ASSETS_DIR / "error_vs_N.html")

print("Saved HTML charts in:", ASSETS_DIR)

Saved HTML charts in: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\03_binomial_discrete_hedging\assets


## 7) Interactive tree visualization (small N)

**Interpretation**
- Node color = option value.
- Hover shows: step, up-moves, stock price, option value, delta.
- Keep N ≤ 12 to stay readable.

In [8]:
def plot_tree_nodes_edges(S: np.ndarray, V: np.ndarray, Delta: np.ndarray, title: str) -> go.Figure:
    N = S.shape[0] - 1

    xs, ys, texts, colors = [], [], [], []
    for i in range(N + 1):
        for j in range(i + 1):
            xs.append(i)
            ys.append(S[i, j])
            val = V[i, j]
            dlt = Delta[i, j] if i < N else np.nan
            texts.append(f"step={i}, up={j}<br>S={S[i,j]:.4f}<br>V={val:.4f}<br>Delta={dlt:.4f}")
            colors.append(val)

    edge_x, edge_y = [], []
    for i in range(N):
        for j in range(i + 1):
            edge_x += [i, i + 1, None]
            edge_y += [S[i, j], S[i + 1, j], None]
            edge_x += [i, i + 1, None]
            edge_y += [S[i, j], S[i + 1, j + 1], None]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=edge_x, y=edge_y, mode="lines", line=dict(width=1), name="edges"))
    fig.add_trace(go.Scatter(
        x=xs, y=ys, mode="markers", name="nodes",
        marker=dict(size=10, color=colors, colorscale="Viridis", showscale=True,
                    colorbar=dict(title="Option value")),
        text=texts, hoverinfo="text"
    ))
    fig.update_layout(title=title, xaxis_title="Time step", yaxis_title="Stock price", template="plotly_dark")
    return fig

In [9]:
N_vis = 10
res_vis = price_european_binomial(S0, K, T, r, sigma, N=N_vis, option_type=option_type)
fig_tree = plot_tree_nodes_edges(res_vis["S"], res_vis["V"], res_vis["Delta"],
                                 title=f"CRR Tree (N={N_vis}) — nodes colored by option value")
fig_tree.show()
fig_tree.write_html(ASSETS_DIR / "tree_visualization.html")
print("Saved:", ASSETS_DIR / "tree_visualization.html")

Saved: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\03_binomial_discrete_hedging\assets\tree_visualization.html


### Tree controls (interactive)

In [10]:
S0_slider = widgets.FloatSlider(value=100.0, min=20, max=200, step=1, description="S0")
K_slider  = widgets.FloatSlider(value=100.0, min=20, max=200, step=1, description="K")
T_slider  = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="T")
r_slider  = widgets.FloatSlider(value=0.03, min=-0.02, max=0.10, step=0.005, description="r")
sig_slider= widgets.FloatSlider(value=0.20, min=0.05, max=0.80, step=0.01, description="sigma")
N_slider  = widgets.IntSlider(value=8, min=1, max=12, step=1, description="N")
type_dd   = widgets.Dropdown(options=["call", "put"], value="call", description="type")
btn       = widgets.Button(description="Update tree", button_style="success")
out_tree  = widgets.Output()

display(widgets.VBox([
    widgets.HBox([S0_slider, K_slider, T_slider]),
    widgets.HBox([r_slider, sig_slider, N_slider, type_dd, btn]),
    out_tree
]))

def update_tree(_):
    with out_tree:
        clear_output(wait=True)
        res = price_european_binomial(float(S0_slider.value), float(K_slider.value), float(T_slider.value),
                                      float(r_slider.value), float(sig_slider.value), int(N_slider.value),
                                      option_type=str(type_dd.value))
        bs = bs_price(float(S0_slider.value), float(K_slider.value), float(T_slider.value),
                      float(r_slider.value), float(sig_slider.value), option_type=str(type_dd.value))
        fig = plot_tree_nodes_edges(res["S"], res["V"], res["Delta"],
                                    title=f"CRR Tree (N={res['N']}) — CRR={res['price']:.4f} | BS={bs:.4f}")
        fig.show()

btn.on_click(update_tree)
update_tree(None)

## 8) Discrete-time delta hedging (BS delta on a GBM path)

We simulate a GBM path and rebalance a BS-delta hedge at discrete times.

Hedging error at maturity:
$$
\text{Error} = \Pi_T - \text{Payoff}(S_T)
$$

**Interpretation**
- Higher rebalancing frequency (larger M) ⇒ smaller replication error.

In [11]:
def simulate_gbm_path(S0: float, T: float, mu: float, sigma: float, M: int, rng: np.random.Generator):
    dt = T / M
    t = np.linspace(0.0, T, M + 1)
    Z = rng.standard_normal(size=M)
    logS = np.empty(M + 1, dtype=float)
    logS[0] = math.log(S0)
    for k in range(M):
        logS[k + 1] = logS[k] + (mu - 0.5 * sigma**2) * dt + sigma * math.sqrt(dt) * Z[k]
    return t, np.exp(logS)

def hedge_one_path_bs_delta(
    S0: float, K: float, T: float, r: float, sigma: float, M: int,
    option_type: str, rng: np.random.Generator, mu_for_path: float | None = None
):
    mu = r if mu_for_path is None else mu_for_path
    t, S = simulate_gbm_path(S0, T, mu, sigma, M, rng)
    dt = T / M

    C0 = bs_price(S0, K, T, r, sigma, option_type)
    delta = bs_delta(S0, K, T, r, sigma, option_type)

    B = C0 - delta * S0  # cash account

    deltas = [delta]
    cashes = [B]

    for k in range(M):
        B *= math.exp(r * dt)  # accrues at r

        tau = T - t[k + 1]
        new_delta = delta if tau <= 1e-12 else bs_delta(float(S[k + 1]), K, tau, r, sigma, option_type)

        d_delta = new_delta - delta
        B -= d_delta * S[k + 1]  # finance re-hedging from cash

        delta = new_delta
        deltas.append(delta)
        cashes.append(B)

    portfolio_T = delta * S[-1] + B
    payoff_T = max(S[-1] - K, 0.0) if option_type == "call" else max(K - S[-1], 0.0)

    return {
        "t": t, "S": S, "deltas": np.array(deltas), "cash": np.array(cashes),
        "portfolio_T": float(portfolio_T), "payoff_T": float(payoff_T),
        "error": float(portfolio_T - payoff_T), "C0": float(C0)
    }

In [12]:
# --- FAST (vectorized) hedging engine ---
# This replaces the slow Python-loop version when you click "Run hedging sim".

def bs_delta_vec(S: np.ndarray, K: float, tau: float, r: float, sigma: float, option_type: str) -> np.ndarray:
    # Vectorized Black-Scholes delta for arrays S and scalar tau.
    if tau <= 1e-12:
        # Expiry limit (weakly defined at-the-money; good enough for sim)
        if option_type == "call":
            return (S > K).astype(float)
        else:  # put
            return (S > K).astype(float) - 1.0

    S = np.asarray(S, dtype=float)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    if option_type == "call":
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1.0


def hedging_error_distribution_fast(
    S0: float, K: float, T: float, r: float, sigma: float, option_type: str,
    M: int, n_paths: int, seed: int = 123, mu_for_path: float | None = None
) -> np.ndarray:
    # Vectorized discrete-time delta hedging simulation under GBM.
    rng_local = np.random.default_rng(seed)
    mu = r if mu_for_path is None else mu_for_path

    dt = T / M
    exp_rdt = np.exp(r * dt)

    # Simulate GBM paths (vectorized)
    Z = rng_local.standard_normal(size=(n_paths, M))
    increments = (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z

    logS = np.empty((n_paths, M + 1), dtype=float)
    logS[:, 0] = np.log(S0)
    logS[:, 1:] = logS[:, [0]] + np.cumsum(increments, axis=1)
    S = np.exp(logS)

    # Self-financing hedge
    C0 = bs_price(S0, K, T, r, sigma, option_type)
    delta = float(bs_delta(S0, K, T, r, sigma, option_type))
    B = np.full(n_paths, C0 - delta * S0, dtype=float)

    for k in range(M):
        B *= exp_rdt
        tau = T - (k + 1) * dt
        new_delta = bs_delta_vec(S[:, k + 1], K, tau, r, sigma, option_type)
        B -= (new_delta - delta) * S[:, k + 1]
        delta = new_delta

    portfolio_T = delta * S[:, -1] + B
    payoff_T = np.maximum(S[:, -1] - K, 0.0) if option_type == "call" else np.maximum(K - S[:, -1], 0.0)
    return portfolio_T - payoff_T

### 8.1 One-path visualization

**Interpretation**
- Delta changes through time, especially near maturity / near strike.
- Replication error occurs because hedging is discrete.

In [13]:
M = 52
path_res = hedge_one_path_bs_delta(S0, K, T, r, sigma, M=M, option_type=option_type,
                                  rng=np.random.default_rng(SEED), mu_for_path=r)

print("C0:", path_res["C0"])
print("Payoff_T:", path_res["payoff_T"])
print("Portfolio_T:", path_res["portfolio_T"])
print("Error:", path_res["error"])

figS = go.Figure()
figS.add_trace(go.Scatter(x=path_res["t"], y=path_res["S"], mode="lines+markers", name="S(t)"))
figS.update_layout(title="One GBM path: S(t)", xaxis_title="t", yaxis_title="S", template="plotly_dark")
figS.show()
figS.write_html(ASSETS_DIR / "one_path_S.html")

figD = go.Figure()
figD.add_trace(go.Scatter(x=path_res["t"], y=path_res["deltas"], mode="lines+markers", name="Delta(t)"))
figD.update_layout(title="Discrete hedging: Delta(t)", xaxis_title="t", yaxis_title="Delta", template="plotly_dark")
figD.show()
figD.write_html(ASSETS_DIR / "one_path_delta.html")

C0: 9.413403383853016
Payoff_T: 17.58772394179836
Portfolio_T: 19.50391476078252
Error: 1.9161908189841625


### 8.2 Error distribution vs rebalancing frequency

**Interpretation**
- As M grows, the histogram tightens and the error standard deviation decreases.

In [14]:
def hedging_error_distribution(
    S0: float, K: float, T: float, r: float, sigma: float, option_type: str,
    M: int, n_paths: int, seed: int = 123, mu_for_path: float | None = None
) -> np.ndarray:
    rng_local = np.random.default_rng(seed)
    errs = np.empty(n_paths, dtype=float)
    for i in range(n_paths):
        errs[i] = hedge_one_path_bs_delta(S0, K, T, r, sigma, M, option_type, rng_local, mu_for_path)["error"]
    return errs

Ms = [5, 10, 21, 52, 252]
n_paths = 3000

df_err = pd.concat(
    [pd.DataFrame({"M": M_, "error": hedging_error_distribution(S0, K, T, r, sigma, option_type, M_, n_paths, SEED, r)})
     for M_ in Ms],
    ignore_index=True
)
df_err.groupby("M")["error"].agg(["mean", "std"]).round(6)

,mean,std
M,,
5,-0.042000,2.969667
10,-0.053499,2.137280
21,-0.046375,1.506436
52,-0.030027,0.944609
252,0.001016,0.445146


In [15]:
# If you restarted the kernel or didn't run the previous cells, rebuild df_err automatically.
if "df_err" not in globals():
    Ms = [5, 10, 21, 52, 252]
    n_paths = 3000
    df_err = pd.concat(
        [
            pd.DataFrame(
                {
                    "M": M_,
                    "error": hedging_error_distribution_fast(
                        S0, K, T, r, sigma, option_type,
                        M=M_, n_paths=n_paths, seed=SEED, mu_for_path=r
                    ),
                }
            )
            for M_ in Ms
        ],
        ignore_index=True,
    )

fig = px.histogram(df_err, x="error", color="M", nbins=80,
                   title="Replication error distribution (BS delta hedging)",
                   template="plotly_dark", barmode="overlay", opacity=0.65)
fig.update_layout(xaxis_title="Error = Portfolio_T − Payoff_T", yaxis_title="Count")
fig.show()
fig.write_html(ASSETS_DIR / "hedging_error_hist.html")

df_std = df_err.groupby("M")["error"].std().reset_index()
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_std["M"], y=df_std["error"], mode="lines+markers", name="std(error)"))
fig2.update_layout(title="Std(error) vs M (log-x)", xaxis_title="M", yaxis_title="Std(error)", template="plotly_dark")
fig2.update_xaxes(type="log")
fig2.show()
fig2.write_html(ASSETS_DIR / "hedging_error_std_vs_M.html")

### Hedging controls (interactive)

In [18]:
M_slider = widgets.IntSlider(value=52, min=2, max=252, step=1, description="M")
paths_slider = widgets.IntSlider(value=2000, min=500, max=8000, step=500, description="n_paths")
type_dd2 = widgets.Dropdown(options=["call", "put"], value=option_type, description="type")
btn2 = widgets.Button(description="Run hedging sim", button_style="warning")
out2 = widgets.Output()

display(widgets.VBox([widgets.HBox([type_dd2, M_slider, paths_slider, btn2]), out2]))

import time

def run_hedge(_):
    with out2:
        clear_output(wait=True)
        M_ = int(M_slider.value)
        n_ = int(paths_slider.value)
        typ = str(type_dd2.value)

        t0 = time.perf_counter()
        errs = hedging_error_distribution_fast(S0, K, T, r, sigma, typ, M_, n_, seed=SEED, mu_for_path=r)
        t1 = time.perf_counter()

        df_ = pd.DataFrame({"error": errs})

        fig = px.histogram(
            df_, x="error", nbins=60,
            title=f"Replication error — M={M_}, paths={n_})",
            template="plotly_dark"
        )
        fig.add_vline(x=float(df_["error"].mean()), line_dash="dash", line_width=3)
        fig.update_layout(showlegend=False, xaxis_title="Error", yaxis_title="Count")
        fig.show()
run_hedge(None)

## 9) Summary (what to say in interviews)
- Built a CRR binomial tree and priced European options via backward induction.
- Computed node deltas and visualized the tree interactively.
- Showed convergence to Black–Scholes as N increases.
- Simulated discrete delta hedging and quantified replication error vs rebalancing frequency.